In [1]:
!pip install sentinelsat

     |████████████████████████████████| 48 kB 3.4 MB/s 


In [2]:
from sentinelsat import SentinelAPI

user = 'asad_007' 
password = '987654321' 
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

In [3]:
!wget https://www.dropbox.com/s/ymxuxpcnj88mlz2/NReserve.zip

--2022-03-16 14:36:03--  https://www.dropbox.com/s/ymxuxpcnj88mlz2/NReserve.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ymxuxpcnj88mlz2/NReserve.zip [following]
--2022-03-16 14:36:03--  https://www.dropbox.com/s/raw/ymxuxpcnj88mlz2/NReserve.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1cf43d5ea3dfb615f318e9e1b3.dl.dropboxusercontent.com/cd/0/inline/Bhm78zpqMXbpX6antq_D9oH1h7od9iLdpJm0ZgtQbZ-m4-PfNp-w71tCMNtojoiCtmMLkSQ7d8INeSEbn3vXcw2rNvZlR8OiaDJ8ge6F3oec_1H51e73v1Ml9H1lKTMhdzRwTYwHEAWVIRHMRumOX_WefqcGjSMew1A6934SgEkTYw/file# [following]
--2022-03-16 14:36:04--  https://uc1cf43d5ea3dfb615f318e9e1b3.dl.dropboxusercontent.com/cd/0/inline/Bhm78zpqMXbpX6antq_D9oH1h7od9iLdpJm0ZgtQbZ-m4-PfNp-w71tCMNtojoiCt

In [4]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 14.5 MB/s 
     |████████████████████████████████| 16.7 MB 50.9 MB/s 
     |████████████████████████████████| 6.3 MB 60.9 MB/s 
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_info
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

During h

In [5]:
import geopandas as gpd
import folium 

nReserve = gpd.read_file('/content/NaturalReserve_Polygon.shp')

m = folium.Map([41.7023292727353, 12.34697305914639], zoom_start=12)
folium.GeoJson(nReserve).add_to(m)
m

ModuleNotFoundError: ignored

In [ ]:
from shapely.geometry import MultiPolygon, Polygon

footprint = None
for i in nReserve['geometry']:
    footprint = i

In [ ]:
products = api.query(footprint,
                     date = ('20190601', '20190626'),
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,10)
                    )

In [ ]:
products_gdf = api.to_geodataframe(products)
products_gdf_sorted = products_gdf.sort_values(['cloudcoverpercentage'], ascending=[True])
products_gdf_sorted

In [ ]:
api.download("16e95baf-bbcb-43fe-ab67-5fecea2ffde5")

In [ ]:
!unzip /content/S2B_MSIL2A_20190605T100039_N0212_R122_T33TTG_20190605T143319.zip -d /content/sample_data

In [ ]:
import rasterio as rio
R10 = '/content/sample_data/S2B_MSIL2A_20190605T100039_N0212_R122_T33TTG_20190605T143319.SAFE/GRANULE/L2A_T33TTG_A011731_20190605T100530/IMG_DATA/R10m'
b4 = rio.open(R10+'/T33TTG_20190605T100039_B04_10m.jp2')
b3 = rio.open(R10+'/T33TTG_20190605T100039_B03_10m.jp2')
b2 = rio.open(R10+'/T33TTG_20190605T100039_B02_10m.jp2')


# Create an RGB image 
with rio.open('/content/sample_data/S2B_MSIL2A_20190605T100039_N0212_R122_T33TTG_20190605T143319.SAFE/GRANULE/L2A_T33TTG_A011731_20190605T100530/IMG_DATA/RGB.tiff','w',driver='Gtiff', width=b4.width, height=b4.height, 
              count=3,crs=b4.crs,transform=b4.transform, dtype=b4.dtypes[0]) as rgb:
    rgb.write(b2.read(1),1) 
    rgb.write(b3.read(1),2) 
    rgb.write(b4.read(1),3) 
    rgb.close()

In [ ]:

import fiona
import rasterio
nReserve_proj = nReserve.to_crs({'init': 'epsg:32633'})

with fiona.open("/content/sample_data/S2B_MSIL2A_20190605T100039_N0212_R122_T33TTG_20190605T143319.SAFE/GRANULE/L2A_T33TTG_A011731_20190605T100530/IMG_DATA/RGB.tiff") as src:
    out_image, out_transform = rio.mask.mask(src, nReserve_proj.geometry,crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
with rasterio.open("RGB_masked.tif", "w", **out_meta) as dest:
    dest.write(out_image)

In [ ]:
from osgeo import gdal
import matplotlib.pyplot as plt


dataset = gdal.Open(r'/content/sample_data/S2B_MSIL2A_20190605T100039_N0212_R122_T33TTG_20190605T143319.SAFE/GRANULE/L2A_T33TTG_A011731_20190605T100530/IMG_DATA/RGB.tiff')


In [ ]:
print(dataset.RasterCount)

In [ ]:
band1 = dataset.GetRasterBand(1) # Red channel
band2 = dataset.GetRasterBand(2) # Green channel
band3 = dataset.GetRasterBand(3) # Blue channel

In [ ]:
b1 = band1.ReadAsArray()
b2 = band2.ReadAsArray()
b3 = band3.ReadAsArray()

In [ ]:
import numpy as np
img = np.dstack((b1, b2, b3))
f = plt.figure()
plt.imshow(img)
plt.savefig('Tiff.png')
plt.show()

In [ ]:
import random

import cv2
from matplotlib import pyplot as plt

import albumentations as A

In [ ]:
def visualize(image):
    # Divide all values by 65535 so we can display the image using matplotlib
    image = image / 65535
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(image)

In [ ]:
# The image is taken from http://www.brucelindbloom.com/index.html?ReferenceImages.html
# © Bruce Justin Lindbloom
image = cv2.imread('/content/sample_data/S2B_MSIL2A_20190605T100039_N0212_R122_T33TTG_20190605T143319.SAFE/GRANULE/L2A_T33TTG_A011731_20190605T100530/IMG_DATA/RGB.tiff', cv2.IMREAD_UNCHANGED)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
visualize(image)

In [ ]:
!pip install earthpy

In [ ]:
from glob import glob
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap
import plotly.graph_objects as go

In [ ]:
np.seterr(divide='ignore', invalid='ignore')
S_sentinel_bands = glob("/content/RGB.tiff")
S_sentinel_bands.sort()

In [ ]:
l = []
for i in S_sentinel_bands:
    with rio.open(i, 'r') as f:
        l.append(f.read(1))
arr_st = np.stack(l)
ep.plot_bands(arr_st, cmap = 'gist_earth', figsize = (150, 10), cols = 100, cbar = False)
plt.show()